In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import utils

In [5]:
# data = utils.load_data('../data/output.csv')

In [6]:
# data['custom_index'] = (data['Rol']).apply(utils.create_index)
# data = data.sort_values(by='FechaHora').reset_index().drop(columns='index')
# data.head(100000)

In [11]:
oferta = utils.load_data('../data/processed_courses.csv')
aps = utils.load_data('../data/AP_data.csv')

In [12]:
oferta

,Curso,NRC,Periodo,Edificio,Salón,Días,Horas,Inscritos
0,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,.centro cívico,.RGD_206-7,L,0630-0750,43
1,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,25454,202420,.centro cívico,.RGD_206-7,I,0630-0750,43
2,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,.Edif. Mario Laserna (ML),.ML_515,I,0800-0920,46
3,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,.Edif. Mario Laserna (ML),.ML_606,L,0800-0920,46
4,ADMI-1102 FUNDAMENTOS DE ADMINISTRACION Y GERE...,11411,202420,.centro cívico,.RGD_112-13,I,0800-0920,46
...,...,...,...,...,...,...,...,...
4452,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,.centro cívico,.RGD_311,V,1400-1650,11
4453,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,.centro cívico,.RGD_311,V,1400-1650,11
4454,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,.centro cívico,.RGD_311,V,1400-1650,11
4455,SPUB-4545 FUNDAMENTOS DEL DERECHO A LA SALUD,66159,202420,.centro cívico,.RGD_311,V,1400-1650,11


In [ ]:
import cv2
import numpy as np
import pytesseract

# Carga toda la carpeta de imagenes (assets) sin subcarpetas
images = utils.load_images('../assets')


for i, image in enumerate(images):

    # Convertir la imagen a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir los límites del color rojo en el espacio HSV
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear máscaras para ambos rangos de rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Unir las dos máscaras para cubrir todas las tonalidades de rojo
    mask = mask1 + mask2

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una lista para almacenar las coordenadas de los cuadrados
    square_coords = []

    # Filtrar solo los contornos que sean cuadrados y que tengan un área significativa
    for contour in contours:
        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar si el contorno tiene 4 lados (es un cuadrado o rectángulo)
        if len(approx) == 4 and cv2.contourArea(contour) > 100:  # Filtrar por área
            square_coords.append(approx)

            # Extraer la región del cuadrado detectado de la imagen original
            x, y, w, h = cv2.boundingRect(approx)
            square_roi = image[y:y + h, x:x + w]

            # Preprocesar la imagen antes de OCR (convertir a gris y aplicar umbral)
            gray_square = cv2.cvtColor(square_roi, cv2.COLOR_BGR2GRAY)

            # Binarizar la imagen (solo si es necesario)
            _, threshold_square = cv2.threshold(gray_square, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            text = pytesseract.image_to_string(threshold_square, config='--psm 7')  # psm 7 para una línea de texto

            # Decidir el color basado en el texto
            if "ML-603" in text.upper():
                color = ( 255, 0, 0) 
            else:
                color = (0, 255, 255) 

            # Dibujar el cuadrado completo con el color basado en el texto
            cv2.fillPoly(image, [approx], color)

            # Poner el texto extraído en la imagen
            center_x = x + w // 2
            center_y = y + h // 2

            text_size = cv2.getTextSize(text.strip(), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_width, text_height = text_size
            text_x = center_x - text_width // 2
            text_y = center_y + text_height // 2
            text_color = (70, 42, 17)
            cv2.putText(image, text.strip(), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, text_color, 5, cv2.LINE_AA)

    # Guardar la imagen con los cuadrados detectados y coloreados
    cv2.imwrite(f'../assets/out/{i}.jpg', image)